In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


import src.models.builders as mb
import src.inference.helpers as ih
import src.data.sanity_checks as sanity_checks
import src.modelling_o2.fit_ho2sat_ar as fit_ho2sat_ar


plotsdir = "../../../../PlotsBreathe/O2_modelling/"

In [2]:
# df = breathe_data.build_O2_FEV1_df()
# Save to excel
# df.to_excel(plotsdir + "Breathe_O2_FEV1.xlsx", index=False)
# Load from excel
df = pd.read_excel(plotsdir + "/Breathe_O2_FEV1.xlsx")
# ID column as type string
df["ID"] = df["ID"].astype(str)
# Date Redocrded as datetime
df["Date Recorded"] = df["Date Recorded"].dt.date
sanity_checks.data_types(df)

In [25]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [4]:
df.describe()

,FEV1,O2 Saturation,ecFEV1,Age,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
count,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000
mean,2.198363,96.966501,2.263423,34.801147,166.247120,3.507772,97.711339,64.659185,62.787907,99.238755
std,0.816148,1.649808,0.822063,10.154773,9.151066,0.649323,0.449598,20.301483,20.271387,1.686950
min,0.490000,76.000000,0.500000,18.000000,143.000000,2.213968,96.975001,15.320382,15.013975,77.519654
25%,1.550000,96.000000,1.610000,27.000000,160.000000,2.979444,97.225960,48.198629,46.536607,98.699543
50%,2.030000,97.000000,2.090000,34.000000,166.000000,3.386997,97.989462,62.880920,61.235539,99.767593
75%,2.760000,98.000000,2.830000,41.000000,173.000000,3.987357,98.114941,77.657779,75.719884,100.255981
max,5.260000,100.000000,5.260000,64.000000,193.000000,5.322753,98.340804,149.505350,149.505350,103.026043


## F1 - Airway resistance vs O2 drop

In [66]:
# Infer airway resistance using the model
for id in df.ID.unique():
# for id in ["101", "102"]:
    df_for_ID = df[df.ID == id].copy().reset_index()
    print(
        f"\nRunning for ID {id}, with {len(df_for_ID)} observations"
    )
    # Take one element of the df for ID at column height
    height = df_for_ID.Height[0]
    sex = df_for_ID.Sex[0]
    hfev1_prior = {
        "type": "default",
        "height": height,
        "age": df_for_ID.Age[0],
        "sex": sex,
    }
    ho2sat_prior = {
        "type": "default",
        "height": height,
        "sex": sex,
    }
    (
        model,
        inf_alg,
        HFEV1,
        ecFEV1,
        _,
        _,
        AR,
    ) = mb.build_FEV1_O2_point_in_time_model(hfev1_prior, ho2sat_prior)

    df_for_ID["ecFEV1 inf mean"] = np.nan
    df_for_ID["FEV1 inf mean"] = np.nan
    fev1_computed = []
    ecfev1_computed = []
    for i in range(len(df_for_ID)):
        fev1_obs = df_for_ID.loc[i, "FEV1"]
        ecfev1_obs = df_for_ID.loc[i, "ecFEV1"]
        if fev1_obs in fev1_computed:
            df_for_ID.loc[i, "FEV1 inf mean"] = df_for_ID.loc[
                df_for_ID["FEV1"] == fev1_obs, "ecFEV1 inf mean"
            ].values[0]
        else: 
            res_ar_for_fev1 = ih.infer(inf_alg, [AR], [[ecFEV1, fev1_obs]], show_progress=False)
            df_for_ID.loc[i, "FEV1 inf mean"] = np.multiply(res_ar_for_fev1.values, AR.bins).sum()
        if ecfev1_obs in ecfev1_computed:
            df_for_ID.loc[i, "ecFEV1 inf mean"] = df_for_ID.loc[
                df_for_ID["ecFEV1"] == ecfev1_obs, "ecFEV1 inf mean"
            ].values[0]
        else:
            res_ar_for_ecfev1 = ih.infer(inf_alg, [AR], [[ecFEV1, ecfev1_obs]], show_progress=False)
            df_for_ID.loc[i, "ecFEV1 inf mean"] = np.multiply(res_ar_for_ecfev1.values, AR.bins).sum()
        fev1_computed = np.append(fev1_computed, fev1_obs)
        ecfev1_computed = np.append(ecfev1_computed, ecfev1_obs)

    # Add to df
    df.loc[df.ID == id, "AR from ecFEV1"] = df_for_ID["ecFEV1 inf mean"].values
    df.loc[df.ID == id, "AR from FEV1"] = df_for_ID["FEV1 inf mean"].values

# IntegrationWarning: The maximum number of subdivisions (50) has been achieved. 
# Happens when the results becomes too close to 0



Running for ID 101, with 816 observations
*** Building FEV1 and O2 point in time model ***
{'mean': 97.2259599226287, 'sigma': 1.0304}
Defining gaussian prior with mu = 97.23, sigma = 1.0304


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:309: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(



Running for ID 102, with 151 observations
*** Building FEV1 and O2 point in time model ***
{'mean': 97.18831609071526, 'sigma': 1.0304}
Defining gaussian prior with mu = 97.19, sigma = 1.0304


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:309: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(



Running for ID 103, with 511 observations
*** Building FEV1 and O2 point in time model ***
{'mean': 98.11494107610714, 'sigma': 1.0304}
Defining gaussian prior with mu = 98.11, sigma = 1.0304


...

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:309: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(



Running for ID 357, with 3 observations
*** Building FEV1 and O2 point in time model ***
{'mean': 98.07729724419369, 'sigma': 1.0304}
Defining gaussian prior with mu = 98.08, sigma = 1.0304


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:309: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(



Running for ID 358, with 11 observations
*** Building FEV1 and O2 point in time model ***
{'mean': 97.1757681467441, 'sigma': 1.0304}
Defining gaussian prior with mu = 97.18, sigma = 1.0304


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:309: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


In [67]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEV1 inf mean,FEV1 inf mean,AR from ecFEV1,AR from FEV1
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,0.61198,0.611980,0.61198,0.611980
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,0.61198,0.640704,0.61198,0.640704
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,0.61198,0.611980,0.61198,0.611980
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,0.61198,0.640704,0.61198,0.640704
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,0.61198,0.611980,0.61198,0.611980


In [12]:
# Read s_AW from excel and add to df
# df.to_excel(f"{plotsdir}airway_resistance_df.xlsx")
# Read from excel
df = pd.read_excel(f"{plotsdir}airway_resistance_df.xlsx", index_col=0)
df.ID = df.ID.astype(str)

In [13]:
# Check that the computed airway resistance makes sense
# Plot ID 101 FEV1 profile with Date Recorded
# '113', '126', '202', '331'
ID = "101"
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["FEV1"],
        mode="markers",
        name="FEV1",
    )
)
# Add trace for predicted FEV1
# fig.add_trace(go.Scatter(x=df[df.ID == "101"]["Date Recorded"], y=df[df.ID == "101"]["Predicted FEV1"], mode="markers", name="Predicted FEV1"))
# Add trace for airway resistance using s_AW
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["AR from FEV1"],
        mode="markers",
        name="Airway Resistance (%)",
    )
)
# fig.add_trace(go.Scatter(x=df[df.ID == ID]["Date Recorded"], y=s_AW[ID], mode="markers", name="Airway Resistance"))

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [14]:
# # 0% airway resistance: FEV1 = Predicted FEV1
# # 25% airway resistance: FEV1 = 0.75 * Predicted FEV1
# # Negative airway resistance: FEV1 > Predicted FEV1
# # Airway resistance = 1 - FEV1/Predicted FEV1
df["Airway Resistance Computed (%)"] = 100 - df["ecFEV1 % Predicted"]

df["Drop from Healthy O2 Saturation (%)"] = (
    df["O2 Saturation"] - df["Healthy O2 Saturation"]
)
df["O2 Saturation % Predicted"] = (
    df["O2 Saturation"] / df["Healthy O2 Saturation"] * 100
)

In [15]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEV1 inf mean,FEV1 inf mean,AR from ecFEV1,AR from FEV1,Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),O2 Saturation % Predicted
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,0.61198,0.611980,0.61198,0.611980,63.435523,-0.22596,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,0.61198,0.640704,0.61198,0.640704,63.435523,-1.22596,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,0.61198,0.611980,0.61198,0.611980,63.435523,-1.22596,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,0.61198,0.640704,0.61198,0.640704,63.158519,-0.22596,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,0.61198,0.611980,0.61198,0.611980,62.327508,0.77404,100.796125


### Raw scatter plot

In [21]:
# Group Airway Resistance Computed (%) into bins of 1% width between 0 and 99
df["Airway Resistance Computed Bins (%)"] = pd.cut(
    df["Airway Resistance Computed (%)"], bins=np.arange(0, 100, 1), include_lowest=False
)
df["Airway Resistance Computed Bins Mean (%)"] = df.groupby("Airway Resistance Computed Bins (%)")[["Airway Resistance Computed (%)"]].transform("mean")

/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_19322/1393740842.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [22]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Airway Resistance computed from ecFEV1 ({df.ID.nunique()} IDs, {len(df)} datapoints)"

# histogram up df["Airway Resistance Computed (%)"] by bins of 1%
# df["Airway Resistance Computed (%)"].hist(bins=100)

fig = px.scatter(
    df,
    x="Airway Resistance Computed (%)",
    # x="Airway Resistance Computed Bins Mean (%)",
    # y="Drop from Healthy O2 Saturation (%)",
    y="O2 Saturation % Predicted",
    title=title,
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.update_layout(font=dict(size=10))
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

In [23]:
df["Airway Resistance from FEV1 (%)"] = df["AR from FEV1"] * 100
df["Airway Resistance from ecFEV1 (%)"] = df["AR from ecFEV1"] * 100

In [25]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Airway Resistance from ecFEV1 ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(
    df,
    # x="Airway Resistance from ecFEV1 binned (%)",
    x="Airway Resistance from ecFEV1 (%)",
    y="O2 Saturation % Predicted",
    # y="Drop from Healthy O2 Saturation (%)",
    title=title,
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

In [113]:
df.sort_values(by="Airway Resistance from ecFEV1 (%)", ascending=True).head(10)

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,...,AR from FEV1,Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),O2 Saturation % Predicted,Airway Resistance Computed Bins (%),Airway Resistance Computed Bins Mean (%),Airway Resistance from FEV1 (%),Airway Resistance from ecFEV1 (%),Airway Resistance from ecFEV1 binned (%),Airway Resistance from FEV1 binned (%)
2816,113,2021-02-04,5.18,99.0,5.26,26,Male,165.5,3.923640,97.320070,...,0.022413,-34.059204,1.679930,101.726191,NaN,NaN,2.241280,2.241280,2,2
2805,113,2021-01-13,5.18,99.0,5.22,26,Male,165.5,3.923640,97.320070,...,0.024251,-33.039743,1.679930,101.726191,NaN,NaN,2.425078,2.241280,2,2
2804,113,2021-01-12,5.22,98.0,5.22,26,Male,165.5,3.923640,97.320070,...,0.022413,-33.039743,0.679930,100.698654,NaN,NaN,2.241280,2.241280,2,2
2803,113,2021-01-11,4.99,98.0,5.22,26,Male,165.5,3.923640,97.320070,...,0.028589,-33.039743,0.679930,100.698654,NaN,NaN,2.858924,2.241280,2,2
2817,113,2021-02-05,5.26,98.0,5.26,26,Male,165.5,3.923640,97.320070,...,0.022413,-34.059204,0.679930,100.698654,NaN,NaN,2.241280,2.241280,2,2
2818,113,2021-02-08,4.87,99.0,5.26,26,Male,165.5,3.923640,97.320070,...,0.031173,-34.059204,1.679930,101.726191,NaN,NaN,3.117251,2.241280,2,3
13693,202,2020-10-09,2.07,99.0,3.31,64,Female,157.0,2.213968,98.165133,...,0.133247,-49.505350,0.834867,100.850472,NaN,NaN,13.324721,2.368655,2,13
13692,202,2020-10-02,3.31,98.0,3.31,64,Female,157.0,2.213968,98.165133,...,0.023687,-49.505350,-0.165133,99.831781,NaN,NaN,2.368655,2.368655,2,2
13691,202,2020-09-25,3.00,99.0,3.31,64,Female,157.0,2.213968,98.165133,...,0.026149,-49.505350,0.834867,100.850472,NaN,NaN,2.614937,2.368655,2,2
2807,113,2021-01-15,4.95,99.0,5.10,26,Male,165.5,3.923640,97.320070,...,0.028589,-29.981358,1.679930,101.726191,NaN,NaN,2.858924,2.425078,2,2


### Bin airway resistance and compute factor profile

In [99]:
import numpy as np
from scipy.interpolate import BSpline, splrep
from scipy.optimize import curve_fit, minimize


# Group by Airway Resistance and take 80th percentile of O2 Sat / Healthy O2 Sat if there are more than 50 observations
def calc_rmax_o2(df_for_AR, y, percentile=80):
    # print("Taking exact percentile")
    # return np.percentile(df_for_AR[y], percentile), len(df_for_AR)
    # Take data between 80 and 90th percentile
    print("Taking percentile avg in range")
    return np.percentile(
        df_for_AR[y],
        range(percentile - 5, percentile + 5),
    ).mean(), len(df_for_AR)


def fit_factor_profile(df_to_fit, x, y):
    x_data = df_to_fit[x].values
    y_data = df_to_fit[y].values

    # Piecewise fit (constant + polynomial)
    def func(x, x0, y0, k1, k2, k3):
        # x0 = 43
        # y0 = df_to_fit[df_to_fit[x] < x0][
        #     o2_col
        # ].mean()

        return np.piecewise(
            x,
            [x <= x0],
            [
                lambda x: y0,
                lambda x: k1 * np.power((x - x0), 3)
                + k2 * np.power((x - x0), 2)
                + k3 * (x - x0)
                + y0,
            ],
        )

    # def objective(params, x, y):
    #     return np.sum((func(x, *params) - y)**2)

    # Enforce monotonicity constraint
    # constraints = ({'type': 'ineq', 'fun': lambda params: np.diff(func(x_data, *params))})

    # Initial guess for parameters
    # initial_guess = [4.34232599e+01, 8.92599726e-01, -3.60069643e-04, 1.56798589e-02, -2.12605357e-01]

    # # Minimize the objective function with the constraint
    # result = minimize(objective, initial_guess, args=(x_data, y_data), constraints=constraints)
    # parameters = result.x

    parameters, covariance = curve_fit(
        func,
        df_to_fit[x].values,
        df_to_fit[y].values,
    )
    print(f"Parameters: {parameters}")
    df_to_fit["Piecewise fit"] = func(x_data, *parameters)

    # Spline fit
    ## Base value for smoothing parameter
    s = df_to_fit.shape[0] - np.sqrt(2 * df_to_fit.shape[0])
    print(f"Smoothing parameter: {s}")
    ### Create a spline representation of the curve
    ### tck-tuple: (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline.
    tck = splrep(
        x_data,
        y_data,
        s=s,
    )
    ### Evalute the spline repr on a new set of points
    df_to_fit["Spline"] = BSpline(*tck)(df_to_fit[x])

    # Mean smoothing
    df_to_fit["Mean Smoothing"] = (
        df_to_fit[y].rolling(5, center=True).mean()
    )
    return df_to_fit


In [108]:
fev1_col = "FEV1"
fev1_col = "ecFEV1"

ar_col = f"Airway Resistance from {fev1_col} (%)"

df[f"Airway Resistance from {fev1_col} binned (%)"] = pd.cut(
    df[f'Airway Resistance from {fev1_col} (%)'], bins=np.arange(0, 100, 1), labels=False
)
# df["Airway Resistance from ecFEV1 binned (%)"] = df["Airway Resistance from ecFEV1 binned (%)"].astype(str)
df[f"Airway Resistance from {fev1_col} binned (%)"] = df[f"Airway Resistance from {fev1_col} binned (%)"] * 1
# Order df by values of Airway Resistance from ecFEV1 binned (%)
# df.sort_values(by="Airway Resistance from ecFEV1 binned (%)")

In [109]:
ar_col_binned = f"Airway Resistance from {fev1_col} binned (%)"

df_for_AW_O2Sat_study = df.copy()
df_for_AW_O2Sat_study = df[~df.ID.isin(["122"])].copy()

for prctile in [10, 85, 90, 95]:  # range(60, 90, 5):
# for prctile in [90]:  # range(60, 90, 5):
    rmax_o2_sat_col = f"{prctile}th-rmax O2 Saturation<br> % Predicted"

    rmax_AW_O2Sat = df_for_AW_O2Sat_study.groupby([ar_col_binned]).apply(
        lambda x: calc_rmax_o2(x, "O2 Saturation % Predicted", prctile)
    )
    # Unstack rmax_AR_O2Sat tuples into 2 columns
    rmax_AW_O2Sat = (
        rmax_AW_O2Sat.apply(pd.Series)
        .rename(columns={0: rmax_o2_sat_col, 1: "#datapoints"})
        .reset_index()
    )
    # Add column for >50 datapoints
    rmax_AW_O2Sat[">50 datapoints"] = rmax_AW_O2Sat["#datapoints"] > 50
    # Mask for >50 datapoints
    rmax_AW_O2Sat_plot = fit_factor_profile(
        rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"]].copy(), 
        x=ar_col_binned,
        y=rmax_o2_sat_col
    )

    # PLot rmax_AW_O2Sat
    title = f"Max achievable O2 Saturation % Predicted ({prctile}th-rmax) vs Airway Resistance ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=rmax_AW_O2Sat_plot[ar_col_binned],
            y=rmax_AW_O2Sat_plot[rmax_o2_sat_col],
            mode="markers",
            name="Airway Resistance vs O2 drop",
        ),
    )
    # fig.add_trace(
    #     go.Scatter(
    #         x=rmax_AW_O2Sat_plot[ar_col_binned],
    #         y=rmax_AW_O2Sat_plot["Spline"],
    #         mode="lines",
    #         name="Spline",
    #     )
    # )
    # fig.add_trace(
    #     go.Scatter(
    #         x=rmax_AW_O2Sat_plot[ar_col_binned],
    #         y=rmax_AW_O2Sat_plot["Piecewise fit"],
    #         mode="markers",
    #         name="Constant + 3rd order polynomial fit",
    #     ),
    # )
    fig.update_traces(line=dict(width=1), marker=dict(size=3))
    fig.update_yaxes(
        range=[90, 103.5],
        nticks=10,
        title=rmax_o2_sat_col,
    )
    fig.update_xaxes(title=ar_col_binned, range=[-5, 91], nticks=10)
    fig.update_layout(title=title, height=300, width=800, font=dict(size=8))
    fig.show()
    # Save to file
    fig.write_image(
        f"{plotsdir}{title}.pdf",
        width=1000,
        height=400,
    )

Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking p

Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking p

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking p

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking percentile avg in range
Taking p

/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning:

Covariance of the parameters could not be estimated



In [35]:
# Bar plot of number of datapoints per airway resistance bin
title = f"Number of datapoints per Airway Resistance bin ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=rmax_AW_O2Sat_plot[ar_col_binned],
        y=rmax_AW_O2Sat_plot["#datapoints"].astype(int),
        name="#datapoints",
        marker=dict(color="grey"),
    ),
)
# Add line for 50 datapoints
fig.add_trace(
    go.Scatter(
        x=rmax_AW_O2Sat[ar_col_binned],
        y=np.repeat(50, len(rmax_AW_O2Sat)),
        mode="lines",
        name="50 datapoints",
    )
)
# Log y axis
fig.update_yaxes(title="#datapoints", nticks=2)
# fig.update_yaxes(type="log", title="#datapoints", nticks=2)
# Range x axis to 91
fig.update_xaxes(range=[-5, 91], nticks=10, title="Binned Airway Resistance (%)")
fig.update_layout(font=dict(size=8), height=250, width=800, title=title)
fig.show()

In [31]:
rmax_AW_O2Sat.sort_values(by="#datapoints", ascending=False).head(10)
rmax_AW_O2Sat_plot.sort_values(by="Airway Resistance from ecFEV1 binned (%)", ascending=False).head(10)

,Airway Resistance from ecFEV1 binned (%),80th-rmax O2 Saturation<br> % Predicted,#datapoints,>50 datapoints,Piecewise fit,Spline,Mean Smoothing
76,78,93.875626,64.0,True,94,95.796427,NaN
69,71,98.602302,87.0,True,97,97.510230,NaN
66,68,98.889212,62.0,True,98,98.095555,98.002900
65,67,98.822189,237.0,True,98,98.272424,99.061898
63,65,99.825172,223.0,True,99,98.600152,98.940245
62,64,99.170613,221.0,True,99,98.751460,99.120647
61,63,97.994039,83.0,True,99,98.894698,99.188575
59,61,99.791221,736.0,True,99,99.157860,99.285950
58,60,99.161829,116.0,True,99,99.278235,99.405346
57,59,100.312046,265.0,True,99,99.391438,99.835555


### Specific cases for the plot with Airway resistance computed

In [273]:
# Plot FEV1 % Predicted with time for individual 122
def plot_fev1_o2(df, ids, o2_col="O2 Saturation"):
    for id in ids:
        df_for_ID = df[df.ID == id]
        # Create subplot with 2 rows
        fig = make_subplots(rows=2, cols=1)
        # Add trace for FEV1 % Predicted on one subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["ecFEV1 % Predicted"],
                mode="markers",
                name="ecFEV1 % Predicted",
            ),
            row=1,
            col=1,
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=df_for_ID["Date Recorded"],
        #         y=df_for_ID["FEV1 % Predicted"],
        #         mode="markers",
        #         name="FEV1 % Predicted",
        #     ),
        #     row=1,
        #     col=1,
        # )
        # Add trace for O2 Saturation on another subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID[o2_col],
                mode="markers",
                name="O2 Saturation",
            ),
            row=2,
            col=1,
        )
        fig.update_traces(marker=dict(size=3), line=dict(width=0.5))
        title = f"ecFEV1 % Predicted and {o2_col} for individual {id} ({len(df_for_ID)} datapoints)"
        fig.update_layout(title=title)
        # Add trace for O2 Saturation on another subplot

        fig.show()

#### Low airway resistance

In [ ]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] < -20]
# '113', '126', '202', '331'

In [ ]:
plot_fev1_o2(df, ["113", "126", "202", "331"])

#### High airway resistance

In [ ]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] > 80].ID.unique()
# 3 individuals '122', '198', '286' have airway resistance > 80%

In [ ]:
# Plot FEV1 % Predicted with time for individual 122
plot_fev1_o2(df, ["122", "286"])

#### High O2 Drop

In [ ]:
df[df["Drop from Healthy O2 Saturation (%)"] < 90].ID.unique()

In [ ]:
plot_fev1_o2(df, ["111", "180", "352"])

#### High positive O2 Drop

In [ ]:
df[df["Drop from Healthy O2 Saturation (%)"] > 2.8].ID.unique()

In [ ]:
df[df.ID == "120"].sort_values(by="Drop from Healthy O2 Saturation (%)")

In [ ]:
plot_fev1_o2(df, ["120"], "Drop from Healthy O2 Saturation (%)")
plot_fev1_o2(df, ["120"])

#### High values (towards 70-72)

In [274]:
ids = ['110', '122', '127', '194', '224', '225']
plot_fev1_o2(df, ids)

In [76]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,...,AR from ecFEV1,AR from FEV1,Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),O2 Saturation % Predicted,Airway Resistance Computed Bins (%),Airway Resistance Computed Bins Mean (%),Airway Resistance from FEV1 (%),Airway Resistance from ecFEV1 (%),Airway Resistance from ecFEV1 binned (%)
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,...,0.61198,0.611980,63.435523,-0.22596,99.767593,"(63, 64]",63.476089,61.197974,61.197974,61
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,...,0.61198,0.640704,63.435523,-1.22596,98.739061,"(63, 64]",63.476089,64.070403,61.197974,64
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,...,0.61198,0.611980,63.435523,-1.22596,98.739061,"(63, 64]",63.476089,61.197974,61.197974,61
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,...,0.61198,0.640704,63.158519,-0.22596,99.767593,"(63, 64]",63.476089,64.070403,61.197974,64
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,...,0.61198,0.611980,62.327508,0.77404,100.796125,"(62, 63]",62.462056,61.197974,61.197974,61


In [79]:
# Amount number of IDs contributing to each bin of Airway Rsistance from ecFEV1 binned (%)
df.groupby("Airway Resistance from ecFEV1 binned (%)").ID.unique()[-10:]

Airway Resistance from ecFEV1 binned (%)
69         [112, 127, 155, 224, 253, 258, 282]
70                        [102, 111, 122, 146]
71    [112, 155, 158, 162, 184, 187, 198, 283]
72              [110, 122, 127, 194, 224, 225]
73              [111, 112, 146, 184, 216, 258]
74    [122, 147, 162, 187, 198, 224, 264, 282]
75              [110, 112, 155, 216, 282, 298]
76                             [112, 122, 146]
77                             [110, 122, 198]
78                                       [286]
Name: ID, dtype: object

In [80]:
df.groupby("Airway Resistance from ecFEV1 binned (%)").ID.unique()[0:10]

Airway Resistance from ecFEV1 binned (%)
2                                            [113, 202]
3                                  [113, 126, 202, 281]
4                   [113, 126, 177, 202, 250, 281, 331]
5         [113, 121, 126, 177, 230, 250, 262, 281, 331]
6     [113, 121, 126, 177, 230, 250, 281, 288, 302, ...
7     [113, 126, 130, 144, 177, 182, 230, 242, 250, ...
8     [107, 113, 126, 130, 141, 144, 177, 182, 190, ...
9     [113, 121, 126, 130, 144, 152, 182, 190, 233, ...
10    [107, 121, 126, 141, 144, 177, 185, 189, 192, ...
11    [113, 126, 130, 141, 152, 170, 177, 182, 189, ...
Name: ID, dtype: object

## F2